In [1]:
import pandas as pd
import numpy as np

In [2]:
def theFunction(p,q,n):
    return p+np.random.rand(n)*(q-p)
theFunction(1,5,3)# from 1 to 5 generate 3 numbers

array([1.27523775, 4.72269045, 2.52237693])

In [3]:
def getCropData(cropName,phmin,phmax,tempmin,tempmax,altmin,altmax,rfmin,rfmax,samplesize=30):
    
    crop= np.repeat(cropName,samplesize).reshape(samplesize,1)
    #ph 0-14
    ph = theFunction(phmin,phmax,samplesize)
    ph=ph.reshape(samplesize,1)   
    ph=np.round(ph,decimals=2) #rounding to 2 decimal places
    #temperature in degree celcius
    temp = np.round(theFunction(tempmin,tempmax,samplesize).reshape(samplesize,1),2)
    altitude = np.round(theFunction(altmin,altmax,samplesize).reshape(samplesize,1),2)
    rainfall = np.round(theFunction(rfmin,rfmax,samplesize).reshape(samplesize,1),2)
    #ph temp altitude rainfall crop
    return np.concatenate([ph,temp,altitude,rainfall,crop],axis=1)

In [4]:
pd.DataFrame(getCropData('RICE',5,6,25,30,0,3000,100,200,10),columns=['ph','temp','altitude','rainfall','crop'])

,ph,temp,altitude,rainfall,crop
0,5.54,28.31,2507.94,170.03,RICE
1,5.04,29.6,2995.95,130.34,RICE
2,5.68,25.94,124.39,182.37,RICE
3,5.45,26.88,1179.18,166.76,RICE
4,5.64,25.61,2335.82,125.76,RICE
5,5.21,27.82,530.69,147.87,RICE
6,5.1,25.32,1642.19,141.74,RICE
7,5.34,26.99,719.38,191.17,RICE
8,5.41,28.33,867.11,169.43,RICE
9,5.02,26.3,2533.69,139.41,RICE


In [19]:
#cropName,phmin,phmax,tempmin,tempmax,altmin,altmax,rfmin,rfmax,samplesize=30
#ricedata=getCropData('RICE',5,6,25,30,0,3000,100,200,200)

maizedata = getCropData('MAIZE',5.5,7,25,30,0,2200,600,900,1000)
paddydata = getCropData('PADDY',6,7,20,25,0,3000,25,150,1000)
milletdata = getCropData('MILLET',5,6,15,20,0,2100,450,650,1000)
barleydata = getCropData('BARLEY',6,7.5,15,17,0.2000,390,430,1000)
cerealsdata = getCropData('CEREAL',5.5,6.5,21,24,1600,2600,200,300 )
oilseeddata=getCropData('OILSEED',4.5,8,18,24,0,2400,25,500)
potatodata=getCropData('POTATO',4.8,5.5,12,25,1500,4200,500,700)
jutedata=getCropData('JUTE',4.8,5.8,24,38,0,1000,800,1000)
pulsedata=getCropData('PULSE',4.5,6.5,20,35,0,2400,100,400)
tobaccodata=getCropData('TOBACCO',5.8,6.5,24,27,0,2000,500,1250)
sugarcanedata=getCropData('SUGARCANE',5,8.5,15,45,0,1000,500,800)
cottondata=getCropData('COTTON',6,6.5,18,30,0,2100,500,700)
cardamomdata=getCropData('CARDAMOM',5.5,6.5,30,80,600,1200,1500,2150)
beandata=getCropData('BEAN',6,6.5,14,32,1000,2100,200,400)
mustarddata=getCropData('MUSTARD',5,5.7,7,25,0,3000,625,1000)


data=np.concatenate([maizedata,paddydata,milletdata,barleydata,cerealsdata,oilseeddata,potatodata,jutedata,pulsedata,tobaccodata,sugarcanedata,cottondata,cardamomdata,beandata,mustarddata],axis=0)

np.random.shuffle(data)
df=pd.DataFrame(data,columns=['ph','temp','altitude','rainfall','crop'])
df.to_csv('crops.csv')

In [8]:
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split

In [9]:
#shaping features and target and splitting data into training and testing sets 
X = data[:,0:4:1] # start:stop:stepsize
Y = data[:,4:]
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.5)

In [10]:
from sklearn.linear_model import LogisticRegression
lrmodel=LogisticRegression()

In [11]:
lrmodel.fit(x_train,y_train)

/home/gg/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/gg/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/gg/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [12]:
linearRegressionScore=lrmodel.score(x_test.astype(np.float64),y_test)
linearRegressionScore

0.9053571428571429

In [13]:
lrmodel.classes_

array(['BARLEY', 'BEAN', 'CARDAMOM', 'CEREAL', 'COTTON', 'JUTE', 'MAIZE',
       'MILLET', 'MUSTARD', 'OILSEED', 'PADDY', 'POTATO', 'PULSE',
       'SUGARCANE', 'TOBACCO'], dtype='<U32')

In [14]:
def giveSuggestions(features,model,numberOfSuggestions=3):
    predictProbabilities=model.predict_proba([features.astype(np.float64)]) 
    indeces=predictProbabilities[0].argsort()[-numberOfSuggestions:][::-1]
    print("Suggestions in decresing of their priorities")
    for i in range(len(indeces)):
        print(i," : ",model.classes_[indeces[i]])


In [15]:
#giveSuggestions(features,trainedModel, no of suggestions of crops from best crops to leat favourable crop)
giveSuggestions(x_test[6],lrmodel,4)
print("\n\n FOR DEBUGGING --Y_TEST VALUE =>",y_test[6])

Suggestions in decresing of their priorities
0  :  MAIZE
1  :  MILLET
2  :  COTTON
3  :  TOBACCO


 FOR DEBUGGING --Y_TEST VALUE => ['MAIZE']


In [24]:
#!pip install --user --upgrade git+https://github.com/jpmml/sklearn2pmml.git

In [16]:
from sklearn.externals import joblib

In [17]:
#exporting the trained model
joblib.dump(lrmodel, 'model.joblib', compress=9)

['model.joblib']